# MIP with pyomo

## References

[1] https://github.com/bruscalia/optimization-demo-files/blob/1fa7a3825421d0b166195d890f2629c576cfbfda/graph-coloring/graph_coloring.ipynb

## Model Formulation

### Sets and Indices

$c \in C \subseteq \{ 0, \ldots, n-1 \}$: Indices and set of colors.
$i,j \in V=\{ 0, \ldots, n-1 \}$: Indices and set of vertices.

### Parameters

$n \in \mathbb{N}$: number of nodes.

$(i,j) \in E \subseteq V^2$: Set of edges.

### Decision Variables

$x_{i, c} \in \{0, 1\} \ \forall \; i \in V, c \in C$: If node $i$ has color $c$ then gets value $1$, otherwise value $0$.

$c_{i} \in \{ 0, \ldots, n-1 \}$: Color of node $i \in V$.

### Objective Function

- **Number of colors**. We want to minimize the number of colors. 

\begin{equation}
\min \sum_{c \in C} y_c
\tag{0}
\end{equation}

where $y_{c} \in \{0, 1\} \ \forall \; c \in C$. If color $c$ is used then gets value $1$, otherwise value $0$.

### Constraints

- **One color for each node**. We assign one and only one color to each node:

\begin{equation}
\sum_{c \in C} x_{i, c} = 1 \quad \forall \; i \in V
\tag{1}
\end{equation}

- **Adjacent vertices**. No two adjacent vertices have the same color:

\begin{equation}
x_{i, c} + x_{j, c} \leq y_{c} \quad \forall \; (i,j) \in E, \forall \; c \in C
\tag{2}
\end{equation}

- **Break symmetry**. Break symmetry by setting a preference order:

\begin{equation}
y_{C_{k-1}} \leq y_{C_{k}} \quad \forall \; k \in \{ 2, ..., \lvert C \rvert \}
\tag{3}
\end{equation}

# Python implementation

## Install pyomo

In [ ]:
!pip install -q pyomo

## Import the libraries

The following code imports the required libraries.

In [1]:
from typing import List, Tuple
from random import sample
import pyomo.environ as pyo
import pandas as pd

ModuleNotFoundError: No module named 'pyomo.environ'

## Definitions

### Restrictions

In [ ]:
# Fill every node with some color
def fill_cstr(model, i):
    return sum(model.x[i, :]) == 1

# Do not repeat colors on edges and color is used
def edge_cstr(model, i, j, c):
    return model.x[i, c] + model.x[j, c] <= model.y[c]

# Break symmetry by setting a preference order
def break_symmetry(model, c):
    if model.C.first() == c:
        return 0 <= model.y[c]
    else:
        c_prev = model.C.prev(c)
        return model.y[c] <= model.y[c_prev]

### Objective function

In [ ]:
# Total number of colors used
def obj(model):
    return sum(model.y[:])

## Color dealing

### Model instantiation

In [ ]:
def build_ilp(
    nodes: List[int],
    colors: List[int],
    edges: List[Tuple[int, int]]
) -> pyo.ConcreteModel:
    """Instantiates pyomo Integer Linear Programming model for the Graph Coloring Problem

    Parameters
    ----------
    nodes : List[int]
        Node indexes

    colors : List[int]
        List of available colors

    edges : List[Tuple[int, int]]
        Connected edges

    Returns
    -------
    pyo.ConcreteModel
        `Concretemodel` of pyomo
    """

    # Create instance
    model = pyo.ConcreteModel()

    # Create sets
    model.C = pyo.Set(initialize=colors)  # Colors
    model.N = pyo.Set(initialize=nodes)  # Nodes
    model.E = pyo.Set(initialize=edges)  # Edges

    # Create variables
    model.x = pyo.Var(model.N, model.C, within=pyo.Binary)
    model.y = pyo.Var(model.C, within=pyo.Binary)

    # Create constraints
    model.fill_cstr = pyo.Constraint(model.N, rule=fill_cstr)
    model.edge_cstr = pyo.Constraint(model.E, model.C, rule=edge_cstr)
    model.break_symmetry = pyo.Constraint(model.C, rule=break_symmetry)

    # Create objective
    model.obj = pyo.Objective(rule=obj)

    return model

### Model definition from data

In [ ]:
def warmstart_from_data(model, nodes, colors):

    color_set=set(colors)

    for n in nodes:
        for c in color_set:
            if c is colors[n]:
                model.x[n, c].value = 1.0
            else:
                model.x[n, c].value = 0.0

    for c in color_set:
        model.y[c].value = 1.0

def ilp_from_data(nodes, colors, edges) -> pyo.ConcreteModel:
    """Instantiates pyomo Integer Linear Programming model for the Graph Coloring Problem

    Parameters
    ----------
    nodes : List of nodes
    colors : List of colors
    edges : List of edges

    Returns
    -------
    pyo.ConcreteModel
        `Concretemodel` of pyomo
    """
    model = build_ilp(nodes, colors, edges)
    warmstart_from_data(model, nodes, colors)
    return model

### Initialize colors

In [ ]:
# Initialize color choice for every node
def initialize_color(total_color_count):

    new_color_list = sample(range(total_color_count), total_color_count)
    return new_color_list

## Create the data

The code below creates the data for the problem.  

### Read the data file

In [ ]:
url = 'https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/data/gc_50_3'
df_data = pd.read_csv(url, sep=" ", header=None)
df_data.head()

In [ ]:
node_count = int(df_data.at[0,0])
edge_count = int(df_data.at[0,1])

print("Number of nodes =", node_count)
print("Number of edges =", edge_count)

edges_list = []
node_set = set()
for i in range(1, edge_count + 1):
    vs = int(df_data.at[i,0])
    ve = int(df_data.at[i,1])
    edges_list.append((vs, ve))
    node_set.add(vs)
    node_set.add(ve)

nodes_list = sorted(node_set)
assert len(nodes_list) == node_count, "Wrong number of nodes specified"


### Read the colors file

In [ ]:
url = 'https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/dsatur/gc_50_3.dsa'
df_colors = pd.read_csv(url, sep=" ", header=None)
df_colors.head()

In [ ]:
init_color_count = int(df_colors.at[0,0])
colors_list = list(map(int, df_colors[1]))


## Solve

In [ ]:
ilp = ilp_from_data(nodes_list, colors_list, edges_list)
opt = pyo.SolverFactory("appsi_highs")
res = opt.solve(ilp)
print(res)


In [ ]:
colors = []
nodes = []
for n in ilp.N:
    nodes.append(n)
    for c in ilp.C:
        if round(ilp.x[n, c].value, ndigits=0) == 1:
            colors.append(c)

solution = []
for col in colors:
    solution.append(col)

solution_node_set = set(solution)
num_colors = len(solution_node_set)


## Prints the solution

Prints the solution in the specified output format

In [ ]:
# prepare the solution in the specified output format
output_data = str(num_colors) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, solution))

print(output_data)

## Visualize the solution

### Import libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

### Generate colors used in nodes

In [ ]:
def generate_colors(num_colors):
    cmap = plt.get_cmap('tab20')  # You can choose a different colormap
    colors = [cmap(i) for i in np.linspace(0, 1, num_colors)]
    return colors

### Create graph 

In [ ]:
node_list = range(node_count)
edge_list = []
for i in range(1, edge_count + 1):
    edge_list.append((df.at[i,0], df.at[i,1]))

# Create a networkx graph from the edges
G = nx.Graph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

### Plot graph

In [ ]:
plot_colors = generate_colors(num_colors)

node_color_list = [plot_colors[color] for color in solution]

# Draw the graph with node colors
pos = nx.spring_layout(G)  # You can use different layout algorithms

# Draw nodes with specified colors
nx.draw(G, pos, with_labels=True, node_size=700, node_color=node_color_list)

# Display the graph
plt.show()

# Shell script execution

## Upload data

In [ ]:
%%shell

wget -nc -P ./data https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/data/data.zip
cd data
unzip data.zip
rm data.zip
ls

In [ ]:
%%shell

wget -nc -P ./dsatur https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/dsatur/dsatur.zip
cd dsatur
unzip dsatur.zip
rm dsatur.zip
ls
cd ..

## Execute solver

In [ ]:
%%shell

wget -nc https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/pyomo/solver.py

mkdir -p pyomo

for file in $(ls ./data/*)
do
  filename="$(basename "$file")"
  python3 solver.py $file ./dsatur/$filename.dsa > ./tabu/$filename.pym
done